# Understand Events and their Effects

Effects are useful to create conditional relationships in your data: if some events happen `Effect`s can be configured to modify vectors in order to reflect the induced change of the event.

In [1]:
%pip install superlinked==12.20.0

In [2]:
from datetime import datetime, timedelta
import pandas as pd
from superlinked import framework as sl

pd.set_option("display.max_colwidth", 100)

# set "NOW" for a fixed data so the notebook runs the same regardless of the date
date_time_obj = datetime(year=2024, month=8, day=7, hour=0, minute=0, second=0)
now_timestamp = int(date_time_obj.timestamp())
EXECUTOR_DATA = {sl.CONTEXT_COMMON: {sl.CONTEXT_COMMON_NOW: now_timestamp}}

## Setting up event schemas

Events generally have:
- `SchemaReference`s: these contain ids that are resolved in the referenced schema. These reflect the items which were constituents of the event.
- event_type as a string: used to group events so that `Effect`s can be applied to a subset of events
- and id of course

In [3]:
class Paragraph(sl.Schema):
    id: sl.IdField
    body: sl.String


class User(sl.Schema):
    id: sl.IdField
    interest: sl.String


class Event(sl.EventSchema):
    id: sl.IdField
    created_at: sl.CreatedAtField
    paragraph: sl.SchemaReference[Paragraph]
    user: sl.SchemaReference[User]
    event_type: sl.String


paragraph = Paragraph()
user = User()
event = Event()

relevance_space = sl.TextSimilaritySpace(
    text=[user.interest, paragraph.body],
    model="sentence-transformers/all-mpnet-base-v2",
)

# weights in effects control importance between events
# effectively doesn't matter if there is only one effect in the index.
event_effects = [
    sl.Effect(
        relevance_space,
        event.user,
        0.8 * event.paragraph,
        event.event_type == "read",
    )
]

# for this index, only initial data of the user will matter as temperature is 0.
index_low_temp = sl.Index(
    spaces=relevance_space, effects=event_effects, temperature=0.0
)
# for this index, initial data and events of the user will matter equally as temperature is 0.5.
index_mid_temp = sl.Index(
    spaces=relevance_space, effects=event_effects, temperature=0.5
)
# high temperature means the emphasis shifts to event data, and the initial vector will matter less.
# Eventually it will not matter when `temperature = 1.0`.
index_hot_temp = sl.Index(
    spaces=relevance_space, effects=event_effects, temperature=1.0
)

**_NOTE 1:_**  Temperature can be any number between 0 and 1 and controls the tradeoff between initial entity vectors and event effects. Its value can be set based on business logic or parameter tuning. `0.5` is a sensible default.

**_NOTE 2:_**  `Index` argument `max_age` defaults to `None` if omitted meaning no restriction. If set, events older than it will be filtered out and will not affect the vector.

**_NOTE 3:_**  `Index` argument `max_count` only takes effect in our batch system.

In [4]:
source_paragraph: sl.InMemorySource = sl.InMemorySource(paragraph)
source_user: sl.InMemorySource = sl.InMemorySource(user)
source_event: sl.InMemorySource = sl.InMemorySource(event)
executor = sl.InMemoryExecutor(
    sources=[source_paragraph, source_user, source_event],
    indices=[index_low_temp, index_mid_temp, index_hot_temp],
)
app = executor.run()

In [5]:
source_paragraph.put(
    [
        {"id": "paragraph-1", "body": "Glorious animals live in the wilderness."},
        {
            "id": "paragraph-2",
            "body": "Growing computation power enables advancements in AI.",
        },
    ]
)

source_user.put([{"id": "user-1", "interest": "I am interested in wild animals."}])

source_event.put(
    [
        {
            "id": "event-1",
            "created_at": int(
                (date_time_obj - timedelta(days=2)).timestamp()
            ),  # 2 days old event
            "paragraph": "paragraph-2",
            "user": "user-1",
            "event_type": "read",
        }
    ]
)

The creation time of events matter - more recent events are more important. The time related modifier of event weights is linearly correlated with the creation time (`created_at` field) of the event and it's relative position between `NOW` and `NOW - max_age`.

## Making the initial vector count more

Setting `temperature` to 0 in `index_low_temp`, the fact the user read a different paragraph (about AI) does not matter - the initial interest in wild animals will prevail. The index is unaffected by the event.

In [6]:
query_low_temp = (
    sl.Query(index_low_temp).find(paragraph).with_vector(user, sl.Param("user_id"))
)

result_low_temp = app.query(
    query_low_temp,
    user_id="user-1",
)

result_low_temp.to_pandas()

,body,id,similarity_score
0,Glorious animals live in the wilderness.,paragraph-1,0.490420
1,Growing computation power enables advancements in AI.,paragraph-2,0.020562


## The power of events

Increasing temperature switches the effect of events on, and shifts the user vector away from the initial interest (wild animals) towards newly read topics (AI). Even though the user expressed interest in wild animals, as an other document about AI was read the preferences shifted towards the actual empirical assessment of their interest. 

In [7]:
query_mid_temp = (
    sl.Query(index_mid_temp).find(paragraph).with_vector(user, sl.Param("user_id"))
)

result_mid_temp = app.query(
    query_mid_temp,
    user_id="user-1",
)

result_mid_temp.to_pandas()

,body,id,similarity_score
0,Growing computation power enables advancements in AI.,paragraph-2,0.714340
1,Glorious animals live in the wilderness.,paragraph-1,0.368305


### Driven fully by events

Setting `temperature` to 1 means similarities are driven entirely by the event data: hence the `1.0` similarity to the read paragraph.

In [8]:
query_hot_temp = (
    sl.Query(index_hot_temp).find(paragraph).with_vector(user, sl.Param("user_id"))
)

result_hot_temp = app.query(
    query_hot_temp,
    user_id="user-1",
)

result_hot_temp.to_pandas()

,body,id,similarity_score
0,Growing computation power enables advancements in AI.,paragraph-2,1.000000
1,Glorious animals live in the wilderness.,paragraph-1,0.035769
